In [1]:
import os
import torch
import numpy as np
import pandas as pd
import sys
import random
import h5py

sys.path.append('../')

from models.mcnnpytorch.src.crowd_count import CrowdCounter
from models.mcnnpytorch.src import network
from models.mcnnpytorch.src.data_loader import ImageDataLoader
from models.mcnnpytorch.src.timer import Timer
from models.mcnnpytorch.src import utils
from models.mcnnpytorch.src.evaluate_model import evaluate_model

In [2]:
try:
    from termcolor import cprint
except ImportError:
    cprint = None


try:
    from pycrayon import CrayonClient
except ImportError:
    CrayonClient = None

def log_print(text, color=None, on_color=None, attrs=None):
    if cprint is not None:
        cprint(text, color=color, on_color=on_color, attrs=attrs)
    else:
        print(text)

In [3]:
method = 'mcnn'
dataset_name = 'shtechA'
output_dir = '../output/mcnnpytorch/saved_models'

train_path = '../data/SHT/part_A_final/train_data/consolidated'
train_gt_path = '../data/SHT/part_A_final/train_data/ground_truth'
val_path = '../data/SHT/part_A_final/test_data/consolidated'
val_gt_path = '../data/SHT/part_A_final/test_data/ground_truth'

In [4]:
#training configuration
start_step = 0
end_step = 20
lr = 0.00001
momentum = 0.9
disp_interval = 2
log_interval = 250

#Tensorboard  config
use_tensorboard = True
save_exp_name = method + '_' + dataset_name + '_' + 'v1'
remove_all_log = False   # remove all historical experiments in TensorBoard
exp_name = None # the previous experiment name in TensorBoard

In [5]:
# tensorboad
use_tensorboard = use_tensorboard and CrayonClient is not None
if use_tensorboard:
    cc = CrayonClient(hostname='127.0.0.1')
    if remove_all_log:
        cc.remove_all_experiments()
    if exp_name is None:    
        exp_name = save_exp_name 
        exp = cc.create_experiment(exp_name)
    else:
        exp = cc.open_experiment(exp_name)

In [6]:
rand_seed = 64678  
if rand_seed is not None:
    np.random.seed(rand_seed)
    torch.manual_seed(rand_seed)
    torch.cuda.manual_seed(rand_seed)

In [7]:
# load net
net = CrowdCounter()
network.weights_normal_init(net, dev=0.01)
net.train()

CrowdCounter(
  (DME): MCNN(
    (branch1): Sequential(
      (0): Conv2d(
        (conv): Conv2d(3, 16, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
        (relu): ReLU(inplace=True)
      )
      (1): Conv2d(
        (conv): Conv2d(16, 32, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
        (relu): ReLU(inplace=True)
      )
      (2): Conv2d(
        (conv): Conv2d(32, 16, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
        (relu): ReLU(inplace=True)
      )
      (3): Conv2d(
        (conv): Conv2d(16, 8, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
        (relu): ReLU(inplace=True)
      )
    )
    (branch2): Sequential(
      (0): Conv2d(
        (conv): Conv2d(3, 20, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
        (relu): ReLU(inplace=True)
      )
      (1): Conv2d(
        (conv): Conv2d(20, 40, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
        (relu): ReLU(inplace=True)
      )
      (2): Conv2d(
        (conv): Conv2d(40, 20, 

In [8]:
params = list(net.parameters())
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, net.parameters()), lr=lr)

if not os.path.exists(output_dir):
    os.mkdir(output_dir)

In [9]:
# training
train_loss = 0
step_cnt = 0
re_cnt = False
t = Timer()
t.tic()

In [10]:
data_loader = ImageDataLoader(train_path, train_gt_path, shuffle=True, pre_load=True)
data_loader_val = ImageDataLoader(val_path, val_gt_path, shuffle=False, pre_load=True)
best_mae = sys.maxsize

Pre-loading the data. This may take a while...
Completed Loading 4 files
Pre-loading the data. This may take a while...
Completed Loading 3 files


In [11]:
for epoch in range(start_step, end_step+1):    
    step = -1
    train_loss = 0
    for blob in data_loader:                
        step = step + 1        
        im_data = blob['data']
        gt_data = blob['gt_density']
        density_map = net(im_data, gt_data)
        loss = net.loss
        train_loss += loss.data
        step_cnt += 1
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if step % disp_interval == 0:            
            duration = t.toc(average=False)
            fps = step_cnt / duration
            gt_count = np.sum(gt_data)    
            density_map = density_map.data.cpu().numpy()
            et_count = np.sum(density_map)
            utils.save_results(im_data,gt_data,density_map, output_dir)
            log_text = 'epoch: %4d, step %4d, Time: %.4fs, gt_cnt: %4.1f, et_cnt: %4.1f' % (epoch,
                step, 1./fps, gt_count,et_count)
            log_print(log_text, color='green', attrs=['bold'])
            re_cnt = True    
    
       
        if re_cnt:                                
            t.tic()
            re_cnt = False

    if (epoch % 2 == 0):
        save_name = os.path.join(output_dir, '{}_{}_{}.h5'.format(method,dataset_name,epoch))
        network.save_net(save_name, net)     
        #calculate error on the validation dataset 
        mae,mse = evaluate_model(save_name, data_loader_val)
        if mae < best_mae:
            best_mae = mae
            best_mse = mse
            best_model = '{}_{}_{}.h5'.format(method,dataset_name,epoch)
        log_text = 'EPOCH: %d, MAE: %.1f, MSE: %0.1f' % (epoch,mae,mse)
        log_print(log_text, color='green', attrs=['bold'])
        log_text = 'BEST MAE: %0.1f, BEST MSE: %0.1f, BEST MODEL: %s' % (best_mae,best_mse, best_model)
        log_print(log_text, color='green', attrs=['bold'])
        if use_tensorboard:
            exp.add_scalar_value('MAE', mae, step=epoch)
            exp.add_scalar_value('MSE', mse, step=epoch)
            exp.add_scalar_value('train_loss', train_loss/data_loader.get_num_samples(), step=epoch)

epoch:    0, step    0, Time: 16.6296s, gt_cnt: 1545.0, et_cnt:  8.2
epoch:    0, step    2, Time: 9.0322s, gt_cnt: 580.0, et_cnt: 20.0


../models/mcnnpytorch/src/network.py:55: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  v = Variable(torch.as_tensor(x).type(dtype), requires_grad = False, volatile = True)


EPOCH: 0, MAE: 497.9, MSE: 645.6
BEST MAE: 497.9, BEST MSE: 645.6, BEST MODEL: mcnn_shtechA_0.h5
epoch:    1, step    0, Time: 10.4750s, gt_cnt: 707.0, et_cnt: 37.1
epoch:    1, step    2, Time: 5.9879s, gt_cnt: 1545.0, et_cnt: 61.0
epoch:    2, step    0, Time: 3.2585s, gt_cnt: 580.0, et_cnt: 62.8
epoch:    2, step    2, Time: 2.9807s, gt_cnt: 707.0, et_cnt: 83.9
EPOCH: 2, MAE: 449.2, MSE: 605.1
BEST MAE: 449.2, BEST MSE: 605.1, BEST MODEL: mcnn_shtechA_2.h5
epoch:    3, step    0, Time: 2.9086s, gt_cnt: 262.0, et_cnt: 95.6
epoch:    3, step    2, Time: 2.2657s, gt_cnt: 580.0, et_cnt: 105.8
epoch:    4, step    0, Time: 2.0353s, gt_cnt: 262.0, et_cnt: 124.1
epoch:    4, step    2, Time: 2.2885s, gt_cnt: 707.0, et_cnt: 143.9
EPOCH: 4, MAE: 402.4, MSE: 568.0
BEST MAE: 402.4, BEST MSE: 568.0, BEST MODEL: mcnn_shtechA_4.h5
epoch:    5, step    0, Time: 2.3165s, gt_cnt: 262.0, et_cnt: 151.9
epoch:    5, step    2, Time: 1.5287s, gt_cnt: 1545.0, et_cnt: 198.5
epoch:    6, step    0, Time: 1